In [1]:
! pip install backtesting
! pip install pandas_ta

You should consider upgrading via the 'c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('see_all_expewriments.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [59]:
import pandas_ta as ta
import pandas as pd

from backbone.backtesting_trader import BacktestingTrader
from backbone.enums import OperationType
import numpy as np
from statsmodels.tsa.filters.hp_filter import hpfilter
from sklearn.metrics import accuracy_score
import yaml

pd.set_option('display.max_columns', None)


In [60]:
ticker = 'USDJPY'

iteration = '2'
root_path = f'./backbone/data/backtest/experiments/{iteration}'
path = 'Mode_train-Model_random_forest-TrainWw_1920-TrainPd_24-TradStgy_ml_strategy-PerFwTg_8-SL_10-RR_4-CloseTime_True-TS_False'
orders = pd.read_csv(os.path.join(root_path, path, 'orders.csv'))
orders['open_time'] = pd.to_datetime(orders['open_time'])
orders['close_time'] = pd.to_datetime(orders['close_time'])
orders = orders[orders.ticker==ticker]

symbols_path = os.path.join('./backbone/data/backtest/symbols', f'{ticker}.csv')
df = pd.read_csv(symbols_path)
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

In [61]:
orders = orders[['open_time', 'close_time', 'open_price','close_price', 'operation_type', 'stop_loss', 'take_profit', 'profit']]

In [63]:
df_merge = pd.merge(
    orders,
    df[['Close']],
    left_on='open_time',
    right_on='Date',
).rename(columns={'Close':'open_price_df'})

df_merge = pd.merge(
    df_merge,
    df[['Close']],
    left_on='close_time',
    right_on='Date',
).rename(columns={'Close':'close_price_df'})

df_merge[['operation_type', 'open_time','close_time','open_price','open_price_df','close_price','close_price_df','stop_loss','take_profit','profit']]

,operation_type,open_time,close_time,open_price,open_price_df,close_price,close_price_df,stop_loss,take_profit,profit
0,OperationType.BUY,2023-01-18 08:00:00,2023-01-18 09:00:00,130.728,130.728,130.628,130.254,130.628,131.128,-10.200
1,OperationType.BUY,2023-02-03 18:00:00,2023-02-06 00:00:00,131.078,131.078,131.478,131.856,130.978,131.478,41.600
2,OperationType.BUY,2023-02-14 19:00:00,2023-02-15 02:00:00,133.082,133.082,132.982,132.715,132.982,133.482,-10.800
3,OperationType.SELL,2023-02-28 22:00:00,2023-03-01 01:00:00,136.131,136.131,136.231,136.344,136.231,135.731,-10.600
4,OperationType.SELL,2023-03-10 16:00:00,2023-03-10 17:00:00,134.833,134.833,134.433,134.299,134.933,134.433,42.800
5,OperationType.SELL,2023-03-10 18:00:00,2023-03-13 00:00:00,135.045,135.045,134.645,134.405,135.145,134.645,44.800
6,OperationType.BUY,2023-04-14 18:00:00,2023-04-17 02:00:00,133.750,133.750,133.990,133.990,133.650,134.150,28.320
7,OperationType.SELL,2023-05-10 18:00:00,2023-05-11 02:00:00,134.527,134.527,134.127,134.099,134.627,134.127,48.400
8,OperationType.BUY,2023-05-16 19:00:00,2023-05-16 20:00:00,136.591,136.591,136.491,136.386,136.491,136.991,-12.900
9,OperationType.BUY,2023-05-16 21:00:00,2023-05-17 05:00:00,136.323,136.323,136.520,136.520,136.223,136.723,25.216


In [70]:
df_merge[
    (df_merge['open_price']!=df_merge['open_price_df']) 
] 

,open_time,close_time,open_price,close_price,operation_type,stop_loss,take_profit,profit,open_price_df,close_price_df


In [69]:
df_merge[
    (df_merge['operation_type']=='OperationType.BUY') 
    & (df_merge['close_price']<df_merge['open_price']) 
    & (df_merge['profit']>=0) 
] 

,open_time,close_time,open_price,close_price,operation_type,stop_loss,take_profit,profit,open_price_df,close_price_df


In [71]:
df_merge[
    (df_merge['operation_type']=='OperationType.SELL') 
    & (df_merge['close_price']>df_merge['open_price']) 
    & (df_merge['profit']>=0) 
] 

,open_time,close_time,open_price,close_price,operation_type,stop_loss,take_profit,profit,open_price_df,close_price_df


In [25]:
orders['operation_type'] = np.where(orders.operation_type=='OperationType.BUY', 1, -1)
orders.loc[df.index, 'operation'] = orders.operation_type
orders.loc[df.index, 'units'] = orders.units

In [26]:
df.fillna(0, inplace=True)

In [6]:
df.operation.value_counts()

operation
 1.0    10
-1.0     9
Name: count, dtype: int64

In [29]:
df.reset_index(inplace=True)

In [30]:
import pandas as pd
def SIDE():
    return df['operation']
    
def UNITS():
    return df['units']

def BBAND():
    return df['upper_bband'], df['lower_bband']

def HP():
    return df['trend']

def SMA20():
    return df['SMA20']

def ADX():
    return df['adx']

In [1]:
from backtesting import Strategy
from backtesting.lib import crossover


class Bband(Strategy):
    max_hold_period = 8
    hold_period = 0

    sl = 20
    rr = 4
    pips = 0.01
    
    def init(self):
        self.side = self.I(SIDE)
        self.hp = self.I(HP)
        self.sma = self.I(SMA20)
        self.adx = self.I(ADX)
        self.units = self.I(UNITS)
        self.upper_bband, self.lower_bband = self.I(BBAND)
    
    def next(self):
        close = self.data.Close[-1]
        if self.trades:
            self.hold_period += 1

            if self.hold_period > self.max_hold_period:
                self.position.close()
                self.hold_period = 0
        
        
        if self.side == 1 and not self.trades:
            sl = close - (self.sl * self.pips)
            tp = close + (self.rr * self.sl * self.pips)

            units = self.units[-1]
            self.buy(sl=sl, tp=tp, size=0.01)

        if self.side == -1 and not self.trades:
            sl = close + (self.sl * self.pips)
            tp = close - (self.rr * self.sl * self.pips)
            units = self.units[-1]
            self.sell(sl=sl, tp=tp, size=-0.01)


c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
from backtesting import Backtest

bt = Backtest(df, Bband, cash=1000, commission=.002)
stats = bt.run()
stats


NameError: name 'df' is not defined

In [35]:
bt.plot()


GridPlot(id='p1490', ...)

In [ ]:
stats._trades